## 0. Imports

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

## 1. Data Preparation

In [5]:
#Load data
X = pd.read_csv("R6_entrada_anterior.csv").to_numpy() #sensors
y = pd.read_csv('R6_salida.csv').set_index("time").to_numpy() #position


#Normalize data
X_scaled = ((X - X.mean(axis=0)) / X.std(axis=0))
X_scaled = X_scaled[:, ~np.isnan(X_scaled).any(axis=0)]
y_scaled = (y - y.mean(axis=0)) / y.std(axis=0)
y_scaled = y_scaled[:, ~np.isnan(y_scaled).any(axis=0)]

#Split into train and test
random_indexes = np.random.permutation(len(X_scaled)) #Randomize batches
split_index = round(len(X_scaled) * 0.8) #Spliing

X_shuffled = X_scaled[random_indexes]
X_train = X_shuffled[:split_index]
X_test = X_shuffled[split_index:]

y_shuffled = y_scaled[random_indexes]
y_train = y_shuffled[:split_index]
y_test = y_shuffled[split_index:]

#Convert into tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

#Print size for checking
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

#Create pythorch class
class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

#Initialize pytorch class
train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset)

torch.Size([27780, 52]) torch.Size([6945, 52]) torch.Size([27780, 10]) torch.Size([6945, 10])


/tmp/ipykernel_11797/2770564947.py:7: RuntimeWarning: divide by zero encountered in divide
  X_scaled = ((X - X.mean(axis=0)) / X.std(axis=0))
/tmp/ipykernel_11797/2770564947.py:7: RuntimeWarning: invalid value encountered in divide
  X_scaled = ((X - X.mean(axis=0)) / X.std(axis=0))


## 2. Create Model

In [6]:
class MLP(nn.Module):
    def __init__(self, input_size:int, output_size:int, hidden_size:int, n_layers:int):
        """
        Inizalization of the MLP model.

        Args:
            input_size (int): Input features (last dimensión of the imput data)
            output_size (int): Output features (last dimensión of the output data)
            hidden_size (int): Number of neurons in the hidding layer (recommended, no more than 128)
            layers (int): Number of layers 
        """
        super(MLP, self).__init__()
        self.hidden_size = hidden_size
        self.layers_list = nn.ModuleList()

        for layer in range(n_layers):
            if layer == 0:
                self.layers_list.append(nn.Linear(input_size, hidden_size))
            elif layer == n_layers -1:
                self.layers_list.append(nn.Linear(hidden_size, output_size))
            else:
                self.layers_list.append(nn.Linear(hidden_size, hidden_size))

    def forward(self, x):

        out = x
        for layer in self.layers_list:
            out = layer(out)

        return out


model = MLP(X_train.shape[1], y_train.shape[1], 32, 4) #32, 4


# Training
def train_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    mean_loss = 0

    for i, batch in enumerate(train_loader):
        X_batch, y_batch = batch[0], batch[1]

        outputs = model(X_batch)
        loss = loss_function(outputs, y_batch)
        mean_loss += loss.item()

        loss.backward() #Calcular los gradientes respecto al loss
        optimizer.step()
        optimizer.zero_grad()

        if (i + 1) % 100 == 0:
            print(f'Batch {i + 1}, Loss: {mean_loss / 100:.3f}')
            mean_loss = 0


def validate_epoch():
    model.train(False)
    mean_loss = 0

    for i, batch in enumerate(test_loader):
        X_batch, y_batch = batch[0], batch[1]

        with torch.no_grad():
            outputs = model(X_batch)
            loss = loss_function(outputs, y_batch)
            mean_loss += loss.item()

    print(f'Validation Loss: {mean_loss / len(test_loader):.3f}')
    print('______________________________________________________________________________')

In [4]:
if __name__ == '__main__':

    # Training Loop
    learning_rate = 1e-3
    epochs = 10
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        train_epoch()
        validate_epoch()


Epoch: 1
Batch 100, Loss: 0.552
Batch 200, Loss: 0.207
Batch 300, Loss: 0.153
Batch 400, Loss: 0.145
Validation Loss: 0.136
______________________________________________________________________________
Epoch: 2
Batch 100, Loss: 0.139
Batch 200, Loss: 0.139
Batch 300, Loss: 0.135
Batch 400, Loss: 0.137
Validation Loss: 0.130
______________________________________________________________________________
Epoch: 3
Batch 100, Loss: 0.134
Batch 200, Loss: 0.135
Batch 300, Loss: 0.131
Batch 400, Loss: 0.134
Validation Loss: 0.127
______________________________________________________________________________
Epoch: 4
Batch 100, Loss: 0.131
Batch 200, Loss: 0.133
Batch 300, Loss: 0.129
Batch 400, Loss: 0.132
Validation Loss: 0.125
______________________________________________________________________________
Epoch: 5
Batch 100, Loss: 0.130
Batch 200, Loss: 0.132
Batch 300, Loss: 0.127
Batch 400, Loss: 0.130
Validation Loss: 0.124
________________________________________________________________

In [ ]:
validate_epoch()

In [7]:
torch.save(model.state_dict(), "MLP_model.pth")